# 1- Lebanese Customs 2011-2019
Date of submission: September 1, 2020.
Version control not applied.

Despite the fact that the free fall of the Lebanese currency exchange rate, has had severe negative impact on the economy, illustrated by the sharp increase in prices, this free fall could create an opportunity for the Lebanese industry to evolve and gain new momentum. In fact, the sharp increase of prices of imported products makes it almost impossible for ordinary consumers to buy as they used to do prior to the crisis, hence the need for cheaper local alternative goods (many of which are virtually inexistent). Also, the decrease of production costs (in US$) of Lebanese goods provides an opportunity for manufacturers to increase their level of exports to foreign markets, an act which would help eliminate gradually trade deficit as new markets start importing cheaper Lebanese goods compared to similar goods’ prices in these markets.

In [1]:
#Hit the 'Run All' Button to start.
#Execution time is at the end of the notebook (Press end button on keyboard for a sneak peek)

# from timeit import default_timer
# start = default_timer()

In [2]:
import pandas as pd
import numpy as np
import glob
import re
import os

## Read csv files of Lebanese Customs data

In [3]:
#I got Lebanese trade data year by year. I have to concatenate them in a single dataframe
filenames = glob.glob('data - lebanese customs/lebanese customs yearly HS6 *.xlsx')
dfs = [pd.read_excel(f, dtype='object') for f in filenames]
df_copy = pd.concat(dfs, ignore_index=True)
del dfs

In [4]:
df = df_copy
print("First few rows")
print("--------------")
print(df.head())
print()
print("Information about columns")
print("-------------------------")
print(df.info())

First few rows
--------------
   year      country  hs_code import_lbp import_usd import_kg   export_lbp  \
0  2011  Afghanistan  0810.20  2,940,783      1,950        33            0   
1  2011  Afghanistan  1602.31          0          0         0   13,860,222   
2  2011  Afghanistan  1602.32          0          0         0  118,593,079   
3  2011  Afghanistan  1602.50          0          0         0  126,046,700   
4  2011  Afghanistan  2202.90          0          0         0   36,008,000   

  export_usd export_kg  
0          0         0  
1      9,191     3,325  
2     78,643    26,337  
3     83,585    25,847  
4     23,878    21,538  

Information about columns
-------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706986 entries, 0 to 706985
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   year        706986 non-null  object
 1   country     706986 non-null  object
 2   hs_code     706986 

In [5]:
#Rename columns
df = df.rename(columns = {'hs_code':'hs6_code'})

## Data Cleaning

### Melting the DataFrame & adding a new column 'Trade Flow'
This move makes it easier for me to filter the data by trade flow when i'm building the dashboard

In [6]:
lebimport = df[['year','country','hs6_code','import_lbp','import_usd','import_kg']]
lebimport.insert(loc = 2, column = 'trade_flow', value = 'import')
lebimport = lebimport.rename(columns = {'import_lbp':'trade_value_lbp',
                                        'import_usd':'trade_value_usd',
                                        'import_kg':'trade_value_kg'})

lebexport = df[['year','country','hs6_code','export_lbp','export_usd','export_kg']]
lebexport.insert(loc = 2, column = 'trade_flow', value = 'export')
lebexport = lebexport.rename(columns = {'export_lbp':'trade_value_lbp',
                                        'export_usd':'trade_value_usd',
                                        'export_kg':'trade_value_kg'})

df = pd.concat([lebimport, lebexport])
del lebimport
del lebexport

### Cleaning Numbers
I read the data with an argument <code>dtype = 'object'</code> and some columns are supposed to contain numerical values. Therefore, they need to be cleaned of commas and dots.

In [7]:
%%time
df['hs6_code'] = df['hs6_code'].str.replace('\.', '', regex=True).astype(np.int64)
df['trade_value_lbp'] = df['trade_value_lbp'].str.replace('\,', '', regex=True).astype(np.int64)
df['trade_value_usd'] = df['trade_value_usd'].str.replace('\,', '', regex=True).astype(np.int64)
df['trade_value_kg'] = df['trade_value_kg'].str.replace('\,', '', regex=True).astype(np.int64)

CPU times: total: 5.11 s
Wall time: 5.2 s


In [8]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1413972 entries, 0 to 706985
Data columns (total 7 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   year             1413972 non-null  object
 1   country          1413972 non-null  object
 2   trade_flow       1413972 non-null  object
 3   hs6_code         1413972 non-null  int64 
 4   trade_value_lbp  1413972 non-null  int64 
 5   trade_value_usd  1413972 non-null  int64 
 6   trade_value_kg   1413972 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 86.3+ MB
None


### Convert HS6 from "As Reported" to "HS92"
The timeframe for this spreadsheet is between 2011 and 2019. During this period, the WCO met twice to amend the Harmonized System by adding, replacing and merging some codes. Therefore we observe three *slightly* different classification methods: H3 (applied between 2007 and 2012), H4 (applied between 2012 and 2017), and H5 (applied between 2017-Present). 

To add some kind of consistency to the nomenclature of commodities, I replaced these three amendments with the original classification H0 (used in 1992). For this, I downloaded three classification conversion csv files (one for each amendment) and used them to map new codes out of the ones I have now. These files are publicly free and available on WCO's official website.

In [9]:
convert_07_92 = pd.read_excel('data - lebanese customs/HS 2007 to HS 1992 Correlation and conversion tables.xls')
convert_12_92 = pd.read_excel('data - lebanese customs/HS 2012 to HS 1992 Correlation and conversion tables.xls')
convert_17_92 = pd.read_excel('data - lebanese customs/HS 2017 to HS 1992 Correlation and conversion tables.xlsx')

In [10]:
print(convert_07_92.head())
print()
print(convert_12_92.head())
print()
print(convert_17_92.head())

    From     To
0  10110  10111
1  10190  10119
2  10210  10210
3  10290  10290
4  10310  10310

   HS 2012  HS 1992
0    10121    10111
1    10129    10119
2    10130    10120
3    10190    10120
4    10221    10210

   From HS 2017  To HS 1992
0         10121       10111
1         10129       10119
2         10130       10111
3         10190       10119
4         10221       10210


<u>Definition of these columns</u>:

I'll use the first dataframe as an example. The first column contains codes to be applied starting 2007, and the second column contains the original version of the first column and was used in 1992. Naming them "From" and "To" respectively, I mean codes in the Lebanese Customs dataframe are in the first column and shall be converted to codes in the second column. The same logic is applied to the other csv files.

In [11]:
convert_12_92.columns = ['From','To']
convert_17_92.columns = ['From','To']

print(convert_07_92.head())
print()
print(convert_12_92.head())
print()
print(convert_17_92.head())

    From     To
0  10110  10111
1  10190  10119
2  10210  10210
3  10290  10290
4  10310  10310

    From     To
0  10121  10111
1  10129  10119
2  10130  10120
3  10190  10120
4  10221  10210

    From     To
0  10121  10111
1  10129  10119
2  10130  10111
3  10190  10119
4  10221  10210


In [12]:
#Create a mapping dictionary out of each file. (i.e. convert dataframes to dictionaries)

conv1 = {}
for pair in convert_07_92.to_dict('records'):
    conv1[pair['From']] = pair['To']
    
conv2 = {}
for pair in convert_12_92.to_dict('records'):
    conv2[pair['From']] = pair['To']

conv3 = {}
for pair in convert_17_92.to_dict('records'):
    conv3[pair['From']] = pair['To']

In [13]:
def convert_code(df):
    """
    This fuction uses two columns of the Lebanese Customs dataframe as inputs: Year, and HS6 Code. 
    Then it checks the year and in which period it should be. We already know there are three possible periods to look 
    for: between 2007 and 2012, between 2012 and 2017, and after 2017. After knowing this information, the fuction 
    converts the code using one of the dictionaries created earlier, depending on the corresponding period. The output
    is the new code.
    Please noted that some codes may not exist in the csv files we just read (i am speaking of codes in chapter 98 which
    is used for national purposes and thus does not have an international definition).
    """
    # Create masks for each year range
    mask_2011 = df['year'] == 2011
    mask_2012_2016 = df['year'].between(2012, 2016)
    mask_2017_2019 = df['year'].between(2017, 2019)

    # Use vectorized operations to update values based on masks
    df['new_hs6_code'] = df['hs6_code']  # Set initial values as the original codes
    df.loc[mask_2011, 'new_hs6_code'] = df.loc[mask_2011, 'hs6_code'].map(conv1.get)
    df.loc[mask_2012_2016, 'new_hs6_code'] = df.loc[mask_2012_2016, 'hs6_code'].map(conv2.get)
    df.loc[mask_2017_2019, 'new_hs6_code'] = df.loc[mask_2017_2019, 'hs6_code'].map(conv3.get)

    # Fill missing values with original codes (might be optimizable, see note below)
    df['new_hs6_code'].fillna(df['hs6_code'], inplace=True)
    df['new_hs6_code'] = df['new_hs6_code'].astype(pd.Int64Dtype())

    return df

In [14]:
%%time
#Apply the covert_code() function.
df = convert_code(df)

#Delete the old HS6 Codes column, and rename the new one.
df = df.drop('hs6_code', axis = 1)\
       .rename(columns = {'new_hs6_code':'hs6_code'})\
       .sort_values(by = ['trade_flow', 'year', 'country', 'hs6_code'])

#Arranging columns in a suitable manner
df = df[['year','country','trade_flow','hs6_code','trade_value_usd','trade_value_lbp','trade_value_kg']]

CPU times: total: 1.48 s
Wall time: 1.5 s


## Get HS1, HS2 and HS4 Codes out of HS6 Codes
HS Codes consist of six digits: the first two digits represent the chapters (HS2 Codes), the second two digits represent the headings in each chapter, and HS4 Codes are the first four digits. 

Sections, or HS1 Codes, are only a distribution of HS2 Codes, and will be manually mapped. 

As a reminder, there are 21 sections that contain 99 chapters. These chapters are subdivided to 1244 heading, which are also subdivided to 5244 subheadings.

In [15]:
#Extract HS2 and HS4 Codes out of HS6 Codes.
df.insert(loc = 3, 
          column = 'hs4_code',
          value = df['hs6_code'].apply(lambda x: np.floor(x/100).astype(np.int64)))

df.insert(loc = 3, 
          column = 'hs2_code', 
          value = df['hs6_code'].apply(lambda x: np.floor(x/10000).astype(np.int64)))

As per WCO, each section contains a list of chapters. For example, section I contains chapter 1 to 5, section II contains chapters 6 to 14, and so on. Here, I make a dictionary in which I pair each section with its list of chapters. This way I can easily map HS1 Codes (sections) using HS2 Codes (chapters) using this dictionary as a reference.

In [16]:
# Mapping HS1 Codes out of HS2 Codes.

section_list = ['I','II','III','IV','V','VI','VII','VIII','IX','X','XI',
                'XII','XIII','XIV','XV','XVI','XVII','XVIII','XIX','XX',
                'XXI']

chapters_list = [[1,2,3,4,5],[6,7,8,9,10,11,12,13,14],[15],
       [16,17,18,19,20,21,22,23,24],[25,26,27],
       [28,29,30,31,32,33,34,35,36,37,38],[39,40],
       [41,42,43],[44,45,46],[47,48,49],
       [50,51,52,53,54,55,56,57,58,59,60,61,62,63], 
       [64,65,66,67],[68,69,70],[71],
       [72,73,74,75,76,77,78,79,80,81,82,83],[84,85],
       [86,87,88,89],[90,91,92],[93],[94,95,96],
       [97,98]]

hs2_hs1 = {}
for section, codelist in zip(section_list, chapters_list):
    for code in codelist:
        hs2_hs1[code] = section
        
df.insert(loc = 3, column = 'hs1_code', value = df['hs2_code'].map(hs2_hs1))

## Add HS Descriptrion
HS is short for Harmonized System. The full name is "Harmonized Commodity Description and Coding System". As the name implies, each commodity is given a unique code, depending on its unique description. Therefore I can easily map commodity descriptions out of codes. For this, I use an excel file that already does this task. This file is publicly free and available on OEC website (short for Observatory of Economic Complexity).

First i read the excel files and convert them into proper dictionaries. Then for each hierarchy level of commodity codes, I map relative descriptions using the dictionaries as reference, and fill any remaining missing value with valid information manually.

In [17]:
hs92 = pd.read_excel('data - lebanese customs/products_hs_92.xlsx', sheet_name = 'products_hs_92')
sections = pd.read_excel('data - lebanese customs/products_hs_92.xlsx', sheet_name = 'sections')

print(hs92.head())
print()
print(sections.head())

    hs92                                    name
0      1                            Live Animals
1    101                                  Horses
2  10111         Horses, live pure-bred breeding
3  10119  Horses, live except pure-bred breeding
4  10120          Asses, mules and hinnies, live

  section                               name
0       I      Live animals; animal products
1      II                 Vegetable products
2     III  Animal or vegetable fats and oils
3      IV                Prepared foodstuffs
4       V                   Mineral products


In [18]:
#Create a mapping dictionary. In other words, convert the pandas dataframes into dictionaries.

get_names_hs92 = {}
for pair in hs92.to_dict('records'):
    get_names_hs92[pair['hs92']] = pair['name']
    
get_names_sections = {}
for pair in sections.to_dict('records'):
    get_names_sections[pair['section']] = pair['name']

### HS6
Mapping the descriptions of HS6 Codes.

In [19]:
df.insert(loc = 7, column = 'hs6_desc', value = df['hs6_code'].map(get_names_hs92)) 

In [20]:
#Check for missing values
pd.isnull(df).sum()

year                  0
country               0
trade_flow            0
hs1_code              0
hs2_code              0
hs4_code              0
hs6_code              0
hs6_desc           3042
trade_value_usd       0
trade_value_lbp       0
trade_value_kg        0
dtype: int64

As mentioned earlier, some codes may not have international definitions. We fill this gap by mannually looking for proper descriptions in the Lebanese customs website.

In [21]:
df[df['hs6_desc'].isnull()]['hs6_code'].unique()

<IntegerArray>
[980100, 980300, 980400, 980200, 710820, 980500]
Length: 6, dtype: Int64

In [22]:
missing = {710820:'Monetary Gold',
           980100:'Used furniture and household appliances and person',
           980200:'Trousseaux of newly-weds and students',
           980300:'Samples of no commercial value',
           980400:'Occasional giftsand personal dispatches',
           980500:'Caskets and coffins containing the body of a deceased'}

df['hs6_desc'] = df['hs6_desc'].fillna(df['hs6_code'].map(missing))


### HS4
Mapping the descriptions of HS4 Codes.

In [23]:
df.insert(loc = 6, column = 'hs4_desc', value = df['hs4_code'].map(get_names_hs92))

In [24]:
#Check for missing values
pd.isnull(df).sum()

year                  0
country               0
trade_flow            0
hs1_code              0
hs2_code              0
hs4_code              0
hs4_desc           3032
hs6_code              0
hs6_desc              0
trade_value_usd       0
trade_value_lbp       0
trade_value_kg        0
dtype: int64

In [25]:
df[df['hs4_desc'].isnull()]['hs4_code'].unique()

array([9801, 9803, 9804, 9802, 9805], dtype=int64)

In [26]:
missing = {9801:'Used furniture and household appliances and person',
           9802:'Trousseaux of newly-weds and students',
           9803:'Samples of no commercial value',
           9804:'Occasional giftsand personal dispatches',
           9805:'Caskets and coffins containing the body of a deceased'}

df['hs4_desc'] = df['hs4_desc'].fillna(df['hs4_code'].map(missing))

### HS2
Mapping the descriptions of HS2 Codes.

In [27]:
df.insert(loc = 5, column = 'hs2_desc', value = df['hs2_code'].map(get_names_hs92))

In [28]:
#Check for missing values
pd.isnull(df).sum()

year                  0
country               0
trade_flow            0
hs1_code              0
hs2_code              0
hs2_desc           3032
hs4_code              0
hs4_desc              0
hs6_code              0
hs6_desc              0
trade_value_usd       0
trade_value_lbp       0
trade_value_kg        0
dtype: int64

In [29]:
df[df['hs2_desc'].isnull()]['hs2_code'].unique()

array([98], dtype=int64)

In [30]:
df['hs2_desc'] = df['hs2_desc'].fillna('Used furniture and household appliances and person')

### HS1
Mapping the descriptions of HS1 Codes.

In [31]:
df.insert(loc = 4, column = 'hs1_desc', value = df['hs1_code'].map(get_names_sections))

In [32]:
#Check for missing values
pd.isnull(df).sum()

year               0
country            0
trade_flow         0
hs1_code           0
hs1_desc           0
hs2_code           0
hs2_desc           0
hs4_code           0
hs4_desc           0
hs6_code           0
hs6_desc           0
trade_value_usd    0
trade_value_lbp    0
trade_value_kg     0
dtype: int64

## Add Regions
Creating a column for regions adds some granularity to the countries with which products are traded. This is useful when it comes to creating a treemap for these countries, because it will allow us to cluster them into their proper regions, adding some beauty to the treemap and helping us analyze it more easily.  

In [33]:
regions = pd.read_excel('data - lebanese customs/country to region.xlsx')
print(regions.head())
print()
print(regions.tail())

          Country          Region
0     Afghanistan  Asia & Pacific
1   Aland Islands          Europe
2         Albania          Europe
3         Algeria     Arab States
4  American Samoa  Asia & Pacific

               Country          Region
243  Wallis and Futuna  Asia & Pacific
244     Western Sahara          Africa
245              Yemen     Middle east
246             Zambia          Africa
247           Zimbabwe          Africa


In [34]:
# Create a map, with countries as key, and corresponding regions as values.
dic = {}
for record in regions.to_dict('records'):
    dic[record['Country']] = record['Region']
    if record['Country'] == 'Bahrain':
        dic[record['Country']] = 'Middle east'
    if record['Region'] == 'Arab States' and record['Country'] != 'Bahrain':
        dic[record['Country']] = 'Africa'


In [35]:
#Insert a new column 'Region'
df.insert(loc = 2, column = 'region', value = df['country'].map(dic))
df.head()

,year,country,region,trade_flow,hs1_code,hs1_desc,hs2_code,hs2_desc,hs4_code,hs4_desc,hs6_code,hs6_desc,trade_value_usd,trade_value_lbp,trade_value_kg
0,2011,Afghanistan,Asia & Pacific,export,II,Vegetable products,8,Fruits and nuts,810,Other Fruits,81020,"Raspberry, blackberry, mulberry and loganberry...",0,0,0
1,2011,Afghanistan,Asia & Pacific,export,IV,Prepared foodstuffs,16,Preparations of meat or fish,1602,Other Prepared Meat,160231,"Turkey meat, offal prepared or preserved, exce...",9191,13860222,3325
2,2011,Afghanistan,Asia & Pacific,export,IV,Prepared foodstuffs,16,Preparations of meat or fish,1602,Other Prepared Meat,160239,"Fowl, duck,goose, offal, prepared, preserved n...",78643,118593079,26337
3,2011,Afghanistan,Asia & Pacific,export,IV,Prepared foodstuffs,16,Preparations of meat or fish,1602,Other Prepared Meat,160250,"Bovine meat, offal nes, not livers, prepared/p...",83585,126046700,25847
4,2011,Afghanistan,Asia & Pacific,export,IV,Prepared foodstuffs,22,Beverages,2202,Flavored Water,220290,"Non-alcoholic beverages nes, except fruit, veg...",23878,36008000,21538


In [36]:
#Check for missing values
pd.isnull(df).sum()

year                   0
country                0
region             78906
trade_flow             0
hs1_code               0
hs1_desc               0
hs2_code               0
hs2_desc               0
hs4_code               0
hs4_desc               0
hs6_code               0
hs6_desc               0
trade_value_usd        0
trade_value_lbp        0
trade_value_kg         0
dtype: int64

In [37]:
#Places with no corresponding regions in the dictionary
sorted(df[df['region'].isnull()]['country'].unique())

['Airline Supplies',
 'Ceuta',
 'Christmas island',
 'Cocos Islands',
 'Duty Free-Airport',
 'East Timor',
 'Falkland Islands',
 'Free Zone',
 'French South Territories',
 'Guinea- Bissau',
 'Heard  Mcdonald Islands',
 'Holy See (Vatican city st',
 'Indian Ocean Territories',
 'Iran, Islamic republic of',
 'Ivory Coast',
 'Kazakstan',
 "Korea,Democ. People's rep",
 'Kosovo',
 "Lao People's Dem Republic",
 'Lebanon-returned Goods',
 'Libyan Arab Jamahiriya',
 'Micronesia,Federation',
 'Misc',
 'Neutral Zone',
 'Northern Mariana islands',
 'Palestine terri, Occupied',
 'Pitcairn',
 'Saint Pierre & Miquelon',
 'Saint Vincent&grenadines',
 'Saint kitts & nevis',
 'Serbia & Montenegro',
 'Ship Supplies',
 'Soolmon Islands',
 'Svalbard&Jan Mayen',
 'Taiwan, Province of china',
 'Tanzania, United Republic',
 'Trinidad & Tobago',
 'Turks&caicos Islands',
 'Us Minor Outlying Islands',
 'Viet Nam',
 'Virgin Island, U.S',
 'Wallis and  Futuna',
 'Warehouse',
 'Yugoslavia',
 'Zaire']

In [38]:
values = {'Duty Free-Airport':'Misc', 
          'Falkland Islands':'South/Latin America', 
          'Free Zone':'Misc',
          'French South Territories':'Misc',
          'Guinea- Bissau':'Africa',
          'Heard  Mcdonald Islands':'Misc', 
          'Holy See (Vatican city st':'Europe',
          'Indian Ocean Territories':'Misc',
          'Iran, Islamic republic of':'Middle east',
          'Ivory Coast':'Africa',
          'Kazakstan':'Asia & Pacific',
          "Korea,Democ. People's rep":'Asia & Pacific',
          'Kosovo':'Europe',
          "Lao People's Dem Republic":'Asia & Pacific',
          'Lebanon-returned Goods':'Misc',
          'Libyan Arab Jamahiriya':'Africa', 
          'Misc':'Misc',
          'Neutral Zone':'Misc',
          'Northern Mariana islands':'Misc',
          'Saint kitts & nevis':'South/Latin America',
          'Serbia & Montenegro':'Europe',
          'Ship Supplies':'Misc',
          'Soolmon Islands':'Asia & Pacific',
          'Svalbard&Jan Mayen':'Europe',
          'Taiwan, Province of china':'Asia & Pacific',
          'Tanzania, United Republic':'Africa',
          'Trinidad & Tobago':'South/Latin America',
          'Turks&caicos Islands':'South/Latin America',
          'Viet Nam':'Asia & Pacific',
          'Virgin Island, U.S':'South/Latin America',
          'Warehouse':'Misc',
          'Yugoslavia':'Europe',
          'Zaire':'Africa',
          'Christmas island':'Misc',
          'Cocos Islands':'Misc',
          'Micronesia,Federation':'Misc',
          'Palestine terri, Occupied':'Middle east',
          'Saint Vincent&grenadines':'South/Latin America',
          'Wallis and  Futuna':'Asia & Pacific',
          'Pitcairn':'Asia & Pacific',
          'Saint Pierre & Miquelon':'North America',
          'East Timor':'Asia & Pacific',
          'Airline Supplies':'Misc',
          'Ceuta':'Europe',
          'Us Minor Outlying Islands':'Misc'}

df['region'] = df['region'].fillna(df['country'].map(values))

# Dash App

In [39]:
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import dash
import dash_daq as daq
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash

C:\Users\user\AppData\Local\Temp\ipykernel_2712\1808612917.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\user\AppData\Local\Temp\ipykernel_2712\1808612917.py:7: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [40]:
# conda install -c conda-forge "dash-bootstrap-components<1"

## Functions to create and update figures

### Function 1: Create a time series (Trade Value over time)

In [41]:
def create_time_series(flow, dff = df):
    #Filter dataframe according to trade flow
    dff = dff[dff['trade_flow'] == flow]
    
    #Style labels on time series:
    def get_text(value):
        '''
        Assign proper labels to points. If a trade value in a certain year is less than one billion,
        it will be written as follow: $123,45M. Otherwise, it will be written as follows: $123,45B, 
        and so on.
        '''
        if value < 10**6:
            text = '${}K'.format(np.round(value/(10**3), 2))
        elif value >=10**6 and value < 10**9:
            text = '${}M'.format(np.round(value/(10**6), 2))
        else:
            text = '${}B'.format(np.round(value/(10**9), 2))
        return text
    
    #Prepare dataframe
    dff = dff.groupby(['year'])['trade_value_usd'].sum().reset_index()
    
    #Create time series
    fig = go.Figure(
        data = go.Scatter(x = dff['year'],
                          y = dff['trade_value_usd'],
                          mode = 'lines+markers+text',
#                           text = dff['Value'],
                          text = [get_text(value) for value in dff['trade_value_usd']],
                          textposition = 'top center',
                          line = dict(color = 'crimson', width = 3),
                          hoverinfo = 'skip'),
        layout = go.Layout(template = 'simple_white',
                           font_family = 'Microsoft Yahei UI',
                           font_size = 13,
                           title = flow,
                           title_font_family = 'Microsoft Yahei UI',
                           title_font_size = 15,
                           title_font_color = 'black')
    )
    
    #Update axes and transition_duration
    fig.update_xaxes(tickfont = dict(family = 'Microsoft Yahei UI', size = 15))
    fig.update_yaxes(tickfont = dict(family = 'Microsoft Yahei UI', size = 15),
                     nticks = 6)
    
    return fig

#Example
#create_time_series('Export', dataframe[dataframe['HS2 Code'] == 15])

### Function 2: Create a treemap for products

In [42]:
def create_products_treemap(flow, year, depth, dff = df):
    #Filter dataframe according to trade flow
    dff = dff[dff['trade_flow'] == flow]
    
    #Filter by year
    if year != 'All years': #A dropdown menu for years will have values 2011, 2012, ..., 2019, and 'All years'.
        dff = dff[dff['year'] == int(year)]
    else:
        dff = dff
        
    #Prepare dataframe
    hierarchy_levels = ['hs1_desc','hs2_desc','hs4_desc','hs6_desc']
    dff = dff.groupby(hierarchy_levels)['trade_value_usd'].sum().reset_index()
    
    #Create treemap
    fig = px.treemap(dff,
                     path = hierarchy_levels,
                     values = 'trade_value_usd',
                     color = hierarchy_levels[0],
                     title = '{}s'.format(flow),
                     maxdepth = int(depth))
    
    fig.update_traces(textinfo = 'label+value',
                      textfont = dict(family = 'Microsoft Yahei UI',
                                      size = 20,
                                      color = 'white'),
                      marker = dict(depthfade = True),
                     hovertemplate = '<b>%{label}</b><br><b>Share: </b> %{percentParent:.2f}')
    
    fig.update_layout(hoverlabel = dict(bgcolor = 'white', font_family = 'Microsoft Yahei UI'),
                      margin = dict(l=20,r=20,b=20),
                      title_font_size = 15,
                      title_font_family = 'Microsoft Yahei UI',
                      title_font_color = 'black')    
    
    return fig

#Example (depth takes the following options: 1,2,3 and 4)
#create_products_treemap('Export', 2019, 2)

### Function 3: Create a treemap for countries

In [43]:
def create_countries_treemap(flow, year, dff = df):
    #Filter dataframe according to trade flow
    dff = dff[dff['trade_flow'] == flow]
    
    #Filter by year
    if year != 'All years': #A dropdown menu for years will have values 2011, 2012, ..., 2019, and 'All years'.
        dff = dff[dff['year'] == int(year)]
        
    #Get title
    title = {'export':'Destinations', 'import':'Origins'}
        
    #Prepare dataframe
    hierarchy_levels = ['region','country']
    dff = dff.groupby(hierarchy_levels)['trade_value_usd'].sum().reset_index()
    
    #Create treemap
    fig = px.treemap(dff,
                     path = hierarchy_levels,
                     values = 'trade_value_usd',
                     color = hierarchy_levels[0],
                     title = title[flow])
    
    fig.update_traces(textinfo = 'label+value',
                      textfont = dict(family = 'Microsoft Yahei UI',
                                      size = 20,
                                      color = 'white'),
                      marker = dict(depthfade = True),
                      hovertemplate = '<b>%{label}</b><br><b>Share: </b> %{percentRoot:.2f}')
    
    fig.update_layout(hoverlabel = dict(bgcolor = 'white', font_family = 'Microsoft Yahei UI'),
                      margin = dict(l=20,r=20,b=20),
                      title_font_size = 15,
                      title_font_family = 'Microsoft Yahei UI',
                      title_font_color = 'black')    
    
    return fig

#Example
#create_countries_treemap('Import',2019)

### Function 4: Create a treemap for products: Growth Change

In [44]:
def build_diff_treemap_products(value, flow, dff = df):        
    y1 = value[0]
    y2 = value[1]
    
    dff = dff[dff['trade_flow'] == flow]
    df1 = dff[dff['year'] == y1]
    df2 = dff[dff['year'] == y2]
    
    hierarchy_levels = ['hs1_desc','hs2_desc','hs4_desc','hs6_desc']
    
    df1 = df1.groupby(hierarchy_levels)['trade_value_usd'].sum().reset_index(name = 'Value at {}'.format(y1))
    df2 = df2.groupby(hierarchy_levels)['trade_value_usd'].sum().reset_index(name = 'Value at {}'.format(y2))
    
    diff = df1.merge(df2, how = 'outer', on = hierarchy_levels)
    
    diff = diff.fillna(0)
    
    diff['Difference'] = diff['Value at {}'.format(y2)] - diff['Value at {}'.format(y1)]
    diff['Absolute'] = diff['Difference'].apply(lambda x: abs(x))
    
    diff = diff[diff['Absolute'] > 0]
    
    title = '{} Difference between {} and {} - Products'.format(flow, y1, y2)
    
    fig = px.treemap(diff,
                     path = hierarchy_levels,
                     values = 'Absolute',
                     color = 'Difference',
                     color_continuous_scale = 'RdBu',
                     color_continuous_midpoint = 0,
                     title = title,
                     maxdepth = 2)
    
    fig.update_traces(textinfo = 'label+value',
                      textfont = dict(family = 'Microsoft Yahei UI',
                                      size = 20),
                      marker = dict(depthfade = True),
                      hovertemplate = '<b>%{label}</b><br><b>Difference: </b>$%{color:.2f}<br>')
    
    fig.update_layout(hoverlabel = dict(bgcolor = 'white', font_family = 'Microsoft Yahei UI'),
                      margin = dict(l=20,r=20,b=20),
                      transition_duration = 500,
                      title_font_size = 15,
                      title_font_family = 'Microsoft Yahei UI',
                      title_font_color = 'black')  
    return fig

#Example
#build_diff_treemap_products([2014,2019], 'Export')

### Function 5: Create a treemap for countries: Growth Change

In [45]:
def build_diff_treemap_countries(value, flow, dff = df):
    y1 = value[0]
    y2 = value[1]
    
    dff = dff[dff['trade_flow'] == flow]
    df1 = dff[dff['year'] == y1]
    df2 = dff[dff['year'] == y2]
    
    hierarchy_levels = ['region','country']
    
    df1 = df1.groupby(hierarchy_levels)['trade_value_usd'].sum().reset_index(name = 'Value at {}'.format(y1))
    df2 = df2.groupby(hierarchy_levels)['trade_value_usd'].sum().reset_index(name = 'Value at {}'.format(y2))
    
    diff = df1.merge(df2, how = 'inner', on = hierarchy_levels)
    
    diff = diff.fillna(0)
    
    diff['Difference'] = diff['Value at {}'.format(y2)] - diff['Value at {}'.format(y1)]
    diff['Absolute'] = abs(diff['Difference'])
    
    diff = diff[diff['Absolute'] > 0]
    
    title = '{} Difference between {} and {} - Countries'.format(flow, y1, y2)
    
    fig = px.treemap(diff,
                     path = hierarchy_levels,
                     values = 'Absolute',
                     color = 'Difference',
                     color_continuous_scale = 'RdBu',
                     color_continuous_midpoint = 0,
                     title = title)
    fig.update_traces(textinfo = 'label+value',
                      textfont = dict(family = 'Microsoft Yahei UI',
                                      size = 20),
                      marker = dict(depthfade = True),
                      hovertemplate = '<b>%{label}</b><br><b>Difference: </b>$%{color:.2f}<br>')
    
    fig.update_layout(hoverlabel = dict(bgcolor = 'white', font_family = 'Microsoft Yahei UI'),
                      margin = dict(l=20,r=20,b=20),
                      transition_duration = 500,
                      title_font_size = 15,
                      title_font_family = 'Microsoft Yahei UI',
                      title_font_color = 'black')  
    return fig

#Example
#build_diff_treemap_countries([2014,2019],'Export')

## Create Pages 

In [46]:
#CSS style for paragraphs
PARAGRAPH_STYLE = {'fontSize':'15px',
                   'text-align':'justify',
                   'text-justify':'inter-word',
                   'padding-right':'15rem'}

### Page 1: Home

In [47]:
### Page 1: Home
home = html.Div([
    dbc.Row(html.H1('Home', className = 'display-1', style = {'padding-left':'15px'})),
    html.Hr(),
    
    dbc.Row(html.H4('Participants in this work', className = 'display-4', style = {'padding-left':'15px'})),
    html.P([html.B('- Prepared by: '), 'Firas Alameddine'], 
           style = PARAGRAPH_STYLE),
    html.P([html.B('- First Supervisor: '), 'Dr. Wissam Sammouri'], 
           style = PARAGRAPH_STYLE),
    html.P([html.B('- Second Supervisor: '), 'Mr. Elias Boustani'], 
           style = PARAGRAPH_STYLE),
    
    html.Br(),
    
    dbc.Row(html.H4('Abstract', className = 'display-4', style = {'padding-left':'15px'})),
    html.P("""Despite the fact that the free fall of the Lebanese currency exchange rate, has had severe negative
    impact on the economy, illustrated by the sharp increase in prices, this free fall could create an opportunity
    for the Lebanese industry to evolve and gain new momentum. In fact, the sharp increase of prices of imported
    products makes it almost impossible for ordinary consumers to buy as they used to do prior to the crisis, hence
    the need for cheaper local alternative goods (many of which are virtually inexistent).
    Also, the decrease of production costs (in US$) of Lebanese goods provides an opportunity for manufacturers
    to increase their level of exports to foreign markets, an act which would help eliminate gradually trade deficit
    as new markets start importing cheaper Lebanese goods compared to similar goods’ prices in these markets.""",
          style = PARAGRAPH_STYLE),
    
    html.Br(),
    
    dbc.Row(html.H4('Project', className = 'display-4', style = {'padding-left':'15px'})),
    html.P([
        html.P("""The goal of this capstone project is to support knowledge acquired throughout my MSBA by getting 
        hand-on work experience with public data. The choice of such a topic is intentional: For it helps me hone my 
        newly acquired skills in data analytics. Accordingly, the progression of my capstone will be divided into the
        following milestones:"""),
        dbc.Row(''),
        html.B('Milestone 1: Use and analyze data in Lebanese Industry and Commerce'),
        html.P("""In the First part, the focus will be on the collection of Lebanese Customs data, as well as 
        international trade data available via official platforms, namely World Bank, WTO and WCO. The aim is to add
        my contribution in terms of reporting methodology, metrics, evaluation, use of visualization, coding 
        automation and recommendations given."""),
        dbc.Row(''),
        html.B('Milestone 2: Data Overview'),
        html.P("""The second part is more informative and will go through an overview of the data tools and sources,
        highlighting both strong and weak points."""),
        html.P(html.I('Tools to be used: Python (main work), Tableau (investigative work).')),
        dbc.Row(''),
        html.B('Milestone 3: Data Driven Answers'),
        html.P('Finally, the third part is meant to provide answers for the following questions:'),
        html.P('''1- What are the main countries Lebanon is dealing with for each import/export category? How did it 
        evolve with time from 2011 to 2019?''', style = {'padding-left':'30px'}),
        html.P("""2- What are the export verticals that are growing in Lebanon and what are the ones shrinking?""",
               style = {'padding-left':'30px'}),
        html.P("""3- How does our pricing per ton for exports compare with the one from other countries?""", 
               style = {'padding-left':'30px'}),
        html.P("""4- Is the pricing per ton for our imports competitive? Are neighboring countries (such as Jordan)
        getting better deals?""", style = {'padding-left':'30px'}),
        html.P("""5- What are the export verticals that exist in Lebanon and that have a potential for growth as 
        other comparable countries were able to grow in the past three years?""", style = {'padding-left':'30px'})
    ], style = PARAGRAPH_STYLE),
    
    html.Br(),
    
    dbc.Row(html.H4('Acknowledgment', className = 'display-4'), style = {'padding-left':'15px'}),
    html.P('''To put everything in a few words, this project is the fruit of all my work for the summer term at AUB MSBA.
    It would be impossible for me to finish it without the guidance and mentorship of Dr Wissam Sammouri and Mr Elias 
    Boustani, and the support of my family and friends.''', style = PARAGRAPH_STYLE)
])

### Page 2: Harmonized System

In [48]:
### Page 2: Harmonized System

harmonized_system = html.Div([
    dbc.Row(html.H1('Harmonized System', className = 'display-1', style = {'padding-left':'15px'})),
    html.Hr(),
    
    dbc.Row(html.H4('Introduction', className ='display-4', style = {'padding-left':'15px'})),
    html.P(html.I(['"With over US$ 15 trillion worth of goods moving around the world, ', 
           html.B('how do you know what is crossing your border? '),
            """This is the issue that the Harmonized Commodity description and Coding System (Commonly known as the
            Harmonized System or the HS) is designed to adress. It is a nomenclature which enables all goods moving
            across borders to be assigned to a class in a uniform manner all over the world. As of 2018. It is used as
            the basis for Customs tariffs and for the compilation of international trade statistics." """], 
          ),style = PARAGRAPH_STYLE),
    
    html.P([html.I('''"The HS is one of the most successful instruments developed by the World Customs Organization (WCO) 
        it adresses a fundamental need of governments: the ability to categorize what is being traded. This enables 
        both decisions on immediate actions for specific goods (duty collections, restrictions or controls, international
        trade regulations) and the use of the collated information to underpin economic and trade related policies
        and planning."'''), """ - World Customs Organization. The Harmonized System: A Universal Language for 
        International Trade. """], style = PARAGRAPH_STYLE),
    
    html.Br(),
    
    dbc.Row(html.H4('Definition',  className ='display-4', style = {'padding-left':'15px'})),
    html.P(["""The Harmonized Commodity Description and Coding System (or simply the Harmonized System or the HS)
    is - as the name implies - a system of codes and names used to categorize and label all sorts of products
    traded around the globe."""], style = PARAGRAPH_STYLE), 
    html.P(["""Since the 1900's, the world has been witnessing an enormous growth in the complexity and range of
    traded products, and in the frequency of trade between parties. With this expansion, the demand of a highly
    structured system aimed at providing accurate information on international trade and detailed descriptions of 
    goods has grown. After several attempts to regularize trade globally, the HS came to existence in 1983 and was put into
    action in 1988. The WCO describes the HS as one of its most successful instruments, because it has been accurately 
    adressing a fundamental need of governments: the categorization of what is traded. As of 2018, the HS is used as 
    a basis for numerous government regulations such as customs tariffs, duty collection, freight taxes, monitoring 
    of controlled goods, and the compilation of international trade statistics, by more than 180 members of the WCO, 
    most of which are developing countries which depend on customs duties for national revenues."""], 
           style = PARAGRAPH_STYLE),
    
    html.P(["""Since its inception, the HS has been revised frequently to overcome disputes rising between trade 
    parties over the right definitions of products, and to stay up to date with new products entering the market 
    and imposing new trade patterns over time. The latest version of the HS was established in 2017, and a newer
    one will take its place in 2022."""],style = PARAGRAPH_STYLE),
    
    html.Br(),
    
    dbc.Row(html.H4('Structure', className = 'display-4', style = {'padding-left':'15px'})),
    html.P(['''The HS uses a hierarchy to categorize traded goods into certain groups. All goods are grouped into 21
    ''', html.I(' Sections '), 'that are subdivided into 96', html.I(' Chapters .'),
    '''Sections and chapters provide broad definitions of products for ease of access and understanding. Furthermore, 
    chapters are subdivided into more than 1200''', html.I(' Headings '), """which are further subdivided into more 
    than 5200""", html.I(' Sub-Headings'), """, to provide a more detailed and accurate decriptions of products."""],
           style = PARAGRAPH_STYLE),
    
    html.P(['''Each product in the Harmonized System is given a 6-digits code. Every member of the WCO 
    (i.e. using the HS) is allowed to do further dissections of product categories, by adding as many digits as they
    feel they need: products with more than 6 digits are said to be at the national tariff-level. To read a product’s
    code, one must bare in mind that the code is actually an assembling of three 2-digits codes: the first couple of
    digits corresponds to chapters (and they go from 1 to 97 with the exclusion of chapter 77), and the second and
    third couples are assigned to headings and sub-headings respectively (coding of headings depends on how
    many headings there is in each chapter. The same analogy is applied to sub-headings relative to headings).'''], 
           style = PARAGRAPH_STYLE),
    
    dbc.Row([
        dbc.Col(
            html.P(['''The photo on the left is an example of what is mentioned right above. Consider the case of
            product 100630: the first couple of digits is 10, referring to chapter 10 in the HS. The second 
            couple is 06, which leads to heading 06 in chapter 10. The third couple is 30, which refers
            to subheading 30 in heading 06. Again, ach section is a list of chapters containing products 
            that are of similar complexity to produce.'''], style = PARAGRAPH_STYLE), width = 8
        ),
        
        dbc.Col(dbc.Card([
            dbc.CardImg(src = '/assets/hs.png', style = {'height':'160px'}),
            dbc.CardFooter('An example of how products are categorized in the HS')
        ]), width = 4)
    ], style = PARAGRAPH_STYLE),
    
    html.Br(),
    
])


### Page 3: User Guide

In [49]:
### Page 3: User Guide

userguide = html.Div([
    dbc.Row(html.H1('User Guide', className = 'display-1', style = {'padding-left':'15px'})),
    html.Hr(),
    
    dbc.Row(html.H4('Introduction', className = 'display-4', style = {'padding-left':'15px'})),
    html.P("""Here, I briefly explain how the dash apps work, by showcasing one component at a time. There are four dash
    apps, each of which has its own purpose, even though they look very similar. Dash App 1 illustrates two treemaps to 
    distribute trade values by products (Imports and Exports) and by countries (Origins and Destinations), and a line
    chart to look at these trade values over time. Mainly, a user would interact with Dash App 1 to observe trade patterns
    of each product between Lebanon and corresponding partners. Dash App 2 works similarly, but its purpose is to observe
    a trade portfolio of products between Lebanon and a certain partner (the other way around, kind of). Dash Apps 3 and 4
    are similar to Dash Apps 1 and 2 (respectively) but they show trade growth change instead of trade values.
    
    Firstly, I define the dropdown menus. Then I explain each figure: What it is, How it works, How to read it and infer
    from it. Lastly, I describe the connection between the figures, by describing how interaction between figures 
    works.""", style = PARAGRAPH_STYLE),
    
    html.Br(),
    
    dbc.Row(html.H4('Dropdown Menus', className = 'display-4', style = {'padding-left':'15px'})),
    dbc.Row([
        dbc.Col(html.P(
            [html.P(["""These dropdown menus are here to filter the database behind the dashoards. They represent the
            categorical fields defining each trade value in the database: commodity description, year and trade flow. 
            The first dropdown menu """, html.B('Select Depth Level '), """implies how deep the treemaps shall be.
            Its first value 1 distributes trade values by sections (first level of granularity in the HS), 
            whereas its second value 2 distributes trade values by sections then chapters 
            (first two levels of granularity in the HS). 

            Toggling between these values change the """, html.Code('maxdepth = ', style = {'fontSize':'15px'}),
            'property in the ' , html.Code('px.treemap() ', style = {'fontSize':'15px'}), """functions. 

            By clicking on the treemap and setting the depth value to 2, the treemap automatically digs into
            the next level. For example, when the user selects a chapter, the treemap immediately shows the headings
            underneath that chapter."""]),
            
            html.P(['The two other Dropdown Menus ', html.B('Select Trade Flow '), 'and ', html.B('Select Year '),
            'filter the dataframe by trade flow and year. The value', html.I(' All years '),
            """in the third menu sums trade values for the whole period between 2011 and 2019.
            The Menus typically activate callbacks each time a value is changed, which hinders time efficiency.
            
            
            Therefore, a """, dbc.Button('Submit', color = 'success', style = {'height':'20px'},
                                         size = 'sm'),
            """ is added to hold these callbacks from firing every single time. A user would then select a filtering
            combination and click on the button to find results he is looking for."""]),
             
             html.P(["""The Range Slider for years is specifically added for Dash Apps 3 and 4, which illustrate the
             growth change of trade values between two years, by products and by countries."""])
        
        ]), width = 8),
        
        dbc.Col(dbc.Card([
            dbc.Label('Select Depth Level'),
            dbc.Select(options=[{'label': str(i), 'value': i} for i in [1, 2]],
                       value=1, size='lg'),

            dbc.Label('Select Trade Flow'),
            dbc.Select(options=[{'label': flow, 'value': flow} for flow in ['export', 'import']],
                       value='export', size='lg'),

            dbc.Label('Select Year'),
            dbc.Select(options=[{'label': 'All years', 'value': 'All years'}] +
                       [{'label': str(year), 'value': year} for year in list(reversed(range(2011, 2020)))],
                       value=1, size='lg'),

            dbc.Label('Select Range of Years'),
            dcc.RangeSlider(marks={year: {'label': str(year)} for year in range(2011, 2020)},
                            value=[2014, 2019], min=2011, max=2019, pushable=True, step=1)
        ], body=True), width=4)
    ], style=PARAGRAPH_STYLE),
    
    html.Br(),
    
    dbc.Row(html.H4('Treemaps', className = 'display-4', style = {'padding-left':'15px'})),
    dbc.Row([
        dbc.Col(dbc.Card(
        dbc.Row([
            dbc.Col(dcc.Loading(dcc.Graph(figure = create_products_treemap('export','All years','2'))), width = 6),
            dbc.Col(dcc.Loading(dcc.Graph(figure = create_countries_treemap('export','All years'))), width = 6)
        ]), body = True
        ), width = {'size':8, 'offset':2})
    ], style = {'padding-bottom':'2rem'}),
    
    
    html.P(["""A treemap is the ideal chart to display tree-structured data, which is the case of bilateral 
    trade data, since the Harmonized System is built around the idea of identifying and distributing products
    into certain groups of different levels, and since countries are also grouped into continents.""",
    """A treemap is a collection of rectangles with different colors and sizes, that are usually ordered by size
    from largest (top left) to smallest (bottom right). Hierarchy of categories in a treemap chart is represented as
    triangles inside other triangles (as countries fall under continents, and chapters fall under sections, etc.).
    The size and place of a rectangle defines how much money is traded between Lebanon and his partners for
    certain product categories."""],style = PARAGRAPH_STYLE),  
    
    html.Br(),
    
    dbc.Row(html.H4('Time Series', className = 'display-4', style = {'padding-left':'15px'})),
    dbc.Row(
        dbc.Col(
            dbc.Card(
                dcc.Graph(
                    figure = create_time_series('import')
                )
                , body = True
            ),width = {'size':8, 'offset':2}
        ), style = {'padding-bottom':'2rem'}
    ),
    html.P(["""The chart above illustrates how Lebanon's total sum of trade value is moving over time. A user will play
    with the dropdown menus to filter the dataframe rendering the line chart. As well, he may click on a treemap 
    and hover over the other one for more detailed filtering and a very narrow view of trade vs time. For example, 
    a user may observe how much worth of gold is exported from Lebanon to Switzerland between 2011 and 2019 using
    Dash App 1, by first setting """, html.B('Select Depth Level'), ' and ', html.B('Select Trade Flow'), ' to ',
    html.I('2'), ' and ', html.I('Export'), """ respectively, then by clicking on 'Gold' rectangle in the first treemap
    and hovering over Switzerland's rectangle in the second treemap."""], style = PARAGRAPH_STYLE)
])

C:\Users\user\anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
C:\Users\user\anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
C:\Users\user\anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
C:\Users\user\anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.c

### Page 4: Dashboards

#### Page 4.1: Dashboard 1
clickData from products treemap & hoverData from countries treemap

In [50]:
dash1 = html.Div([
    dbc.Row([
        dbc.Col([
            dbc.Label('Select HS Level', style = {'fontSize':'15px'}),
            dbc.Select(id = 'hs 1',
                       options = [{'label':'1','value':1},
                                  {'label':'2','value':2}],
                       value = 2,
                       size = 'lg')
        ],width = 2),
        
        dbc.Col([
            dbc.Label('Select Trade Flow', style = {'fontSize':'15px'}),
            dbc.Select(id = 'flow 1',
                       options = [{'label':'export','value':'export'},
                                  {'label':'import','value':'import'}],
                       value = 'export',
                       size = 'lg')
        ], width = 2),
        
        dbc.Col([
            dbc.Label('Select Year', style = {'fontSize':'15px'}),
            dbc.Select(id = 'year 1',
                       options = [{'label':'All years','value':'All years'},
                                  {'label':'2019', 'value':2019},
                                  {'label':'2018', 'value':2018},
                                  {'label':'2017', 'value':2017},
                                  {'label':'2016', 'value':2016},
                                  {'label':'2015', 'value':2015},
                                  {'label':'2014', 'value':2014},
                                  {'label':'2013', 'value':2013},
                                  {'label':'2012', 'value':2012},
                                  {'label':'2011', 'value':2011}],
                       value = 'All years',
                       size = 'lg')
        ], width = 2),
        
        dbc.Col([
            dbc.Label('Select Range of Years', style = {'fontSize':'15px'}),
            dcc.RangeSlider(id = 'slider 1', min = 2011, max = 2019, step = 1, value = [2014,2019], pushable = True,
                            marks = {year:{'label':str(year)} for year in range(2011,2020)}, disabled = True)
        ], width = 5),
        
        dbc.Col([
            dbc.Row(dbc.Button('Submit', id = 'submit 1', color = 'success', n_clicks = 0))
        ], width = 1, style = {'padding-right':'2rem'})
    ], style = {'padding-bottom':'1rem', 'padding-top':'1rem'}),
    
    dbc.Row([
        dbc.Col(dbc.Card(dcc.Loading(dcc.Graph(id = 'treemap 1_1'), type = 'circle'), body = True), width = 6),
        dbc.Col(dbc.Card(dcc.Loading(dcc.Graph(id = 'treemap 1_2', clear_on_unhover = True), type = 'circle'),
                         body = True), width = 6)
    ], style = {'padding-bottom':'1rem'}),
    
    dbc.Row(dbc.Col(dbc.Card(dcc.Loading(dcc.Graph(id = 'time series 1'), type = 'circle'), body = True), width = 12))
    
])

#### Page 4.2: Dashboard 2
clickData from countries treemap & hoverData from products treemap

In [51]:
dash2 = html.Div([
    dbc.Row([
        dbc.Col([
            dbc.Label('Select HS Level', style = {'fontSize':'15px'}),
            dbc.Select(id = 'hs 2',
                       options = [{'label':'1','value':1},
                                  {'label':'2','value':2}],
                       value = 2,
                       size = 'lg')
        ],width = 2),
        
        dbc.Col([
            dbc.Label('Select Trade Flow', style = {'fontSize':'15px'}),
            dbc.Select(id = 'flow 2',
                       options = [{'label':'export','value':'export'},
                                  {'label':'import','value':'import'}],
                       value = 'export',
                       size = 'lg')
        ], width = 2),
        
        dbc.Col([
            dbc.Label('Select Year', style = {'fontSize':'15px'}),
            dbc.Select(id = 'year 2',
                       options = [{'label':'All years','value':'All years'},
                                  {'label':'2019', 'value':2019},
                                  {'label':'2018', 'value':2018},
                                  {'label':'2017', 'value':2017},
                                  {'label':'2016', 'value':2016},
                                  {'label':'2015', 'value':2015},
                                  {'label':'2014', 'value':2014},
                                  {'label':'2013', 'value':2013},
                                  {'label':'2012', 'value':2012},
                                  {'label':'2011', 'value':2011}],
                       value = 'All years',
                       size = 'lg')
        ], width = 2),
        
        dbc.Col([
            dbc.Label('Select Range of Years', style = {'fontSize':'15px'}),
            dcc.RangeSlider(id = 'slider 2', min = 2011, max = 2019, step = 1, value = [2014,2019], pushable = True,
                            marks = {year:{'label':str(year)} for year in range(2011,2020)}, disabled = True)
        ], width = 5),
        
        dbc.Col([
            dbc.Row(dbc.Button('Submit', id = 'submit 2', color = 'success', n_clicks = 0))
        ], width = 1, style = {'padding-right':'2rem'})
    ], style = {'padding-bottom':'1rem', 'padding-top':'1rem'}),
    
    dbc.Row([
        dbc.Col(dbc.Card(dcc.Loading(dcc.Graph(id = 'treemap 2_1'), type = 'circle'), body = True), width = 6),
        dbc.Col(dbc.Card(dcc.Loading(dcc.Graph(id = 'treemap 2_2', clear_on_unhover = True), type = 'circle'),
                         body = True), width = 6)
    ], style = {'padding-bottom':'1rem'}),
    
    dbc.Row(dbc.Col(dbc.Card(dcc.Loading(dcc.Graph(id = 'time series 2'), type = 'circle'), body = True), width = 12))
    
])

#### Page 4.3: Dashboard 3

In [52]:
dash3 = html.Div([
    dbc.Row([
        dbc.Col([
            dbc.Label('Select HS Level', style = {'fontSize':'15px'}),
            dbc.Select(id = 'hs 3',
                       options = [{'label':'1','value':1},
                                  {'label':'2','value':2}],
                       value = 2,
                       size = 'lg')
        ],width = 2),
        
        dbc.Col([
            dbc.Label('Select Trade Flow', style = {'fontSize':'15px'}),
            dbc.Select(id = 'flow 3',
                       options = [{'label':'export','value':'export'},
                                  {'label':'import','value':'import'}],
                       value = 'export',
                       size = 'lg')
        ], width = 2),
        
        dbc.Col([
            dbc.Label('Select Year', style = {'fontSize':'15px'}),
            dbc.Select(id = 'year 3',
                       options = [{'label':'All years','value':'All years'},
                                  {'label':'2019', 'value':2019},
                                  {'label':'2018', 'value':2018},
                                  {'label':'2017', 'value':2017},
                                  {'label':'2016', 'value':2016},
                                  {'label':'2015', 'value':2015},
                                  {'label':'2014', 'value':2014},
                                  {'label':'2013', 'value':2013},
                                  {'label':'2012', 'value':2012},
                                  {'label':'2011', 'value':2011}],
                       value = 'All years',
                       size = 'lg', disabled = True)
        ], width = 2),
        
        dbc.Col([
            dbc.Label('Select Range of Years', style = {'fontSize':'15px'}),
            dcc.RangeSlider(id = 'slider 3', min = 2011, max = 2019, step = 1, value = [2014,2019], pushable = True,
                            marks = {year:{'label':str(year)} for year in range(2011,2020)})
        ], width = 5),
        
        dbc.Col([
            dbc.Row(dbc.Button('Submit', id = 'submit 3', color = 'success', n_clicks = 0))
        ], width = 1, style = {'padding-right':'2rem'})
    ], style = {'padding-bottom':'1rem', 'padding-top':'1rem'}),
    
    dbc.Row([
        dbc.Col(dbc.Card(dcc.Loading(dcc.Graph(id = 'treemap 3_1'), type = 'circle'), body = True), width = 6),
        dbc.Col(dbc.Card(dcc.Loading(dcc.Graph(id = 'treemap 3_2', clear_on_unhover = True), type = 'circle' ),
                         body = True), width = 6)
    ], style = {'padding-bottom':'1rem'}),
    
    dbc.Row(dbc.Col(dbc.Card(dcc.Loading(dcc.Graph(id = 'time series 3'), type = 'circle'), body = True), width = 12))
    
])

#### Page 4.4: Dashboard 4

In [53]:
dash4 = html.Div([
    dbc.Row([
        dbc.Col([
            dbc.Label('Select HS Level', style = {'fontSize':'15px'}),
            dbc.Select(id = 'hs 4',
                       options = [{'label':'1','value':1},
                                  {'label':'2','value':2}],
                       value = 2,
                       size = 'lg')
        ],width = 2),
        
        dbc.Col([
            dbc.Label('Select Trade Flow', style = {'fontSize':'15px'}),
            dbc.Select(id = 'flow 4',
                       options = [{'label':'export','value':'export'},
                                  {'label':'import','value':'import'}],
                       value = 'xport',
                       size = 'lg')
        ], width = 2),
        
        dbc.Col([
            dbc.Label('Select Year', style = {'fontSize':'15px'}),
            dbc.Select(id = 'year 4',
                       options = [{'label':'All years','value':'All years'},
                                  {'label':'2019', 'value':2019},
                                  {'label':'2018', 'value':2018},
                                  {'label':'2017', 'value':2017},
                                  {'label':'2016', 'value':2016},
                                  {'label':'2015', 'value':2015},
                                  {'label':'2014', 'value':2014},
                                  {'label':'2013', 'value':2013},
                                  {'label':'2012', 'value':2012},
                                  {'label':'2011', 'value':2011}],
                       value = 'All years',
                       size = 'lg', disabled = True)
        ], width = 2),
        
        dbc.Col([
            dbc.Label('Select Range of Years', style = {'fontSize':'15px'}),
            dcc.RangeSlider(id = 'slider 4', min = 2011, max = 2019, step = 1, value = [2014,2019], pushable = True,
                            marks = {year:{'label':str(year)} for year in range(2011,2020)})
        ], width = 5),
        
        dbc.Col([
            dbc.Row(dbc.Button('Submit', id = 'submit 4', color = 'success', n_clicks = 0))
        ], width = 1, style = {'padding-right':'2rem'})
    ], style = {'padding-bottom':'1rem', 'padding-top':'1rem'}),
    
    dbc.Row([
        dbc.Col(dbc.Card(dcc.Loading(dcc.Graph(id = 'treemap 4_1'), type = 'circle'), body = True), width = 6),
        dbc.Col(dbc.Card(dcc.Loading(dcc.Graph(id = 'treemap 4_2', clear_on_unhover = True), type = 'circle'),
                         body = True), width = 6)
    ], style = {'padding-bottom':'1rem'}),
    
    dbc.Row(dbc.Col(dbc.Card(dcc.Loading(dcc.Graph(id = 'time series 4'), type = 'circle'), body = True), width = 12))
    
])

#### Tabs to render dashboards

In [54]:
dash = html.Div([
    dbc.Row(html.H1('Dash App', className = 'display-1', style = {'padding-left':'15px'})),
    html.Hr(),
    dbc.Tabs([
        dbc.Tab(dash1, label = 'Dash App 1', label_style = {'fontSize':'15px'}),
        dbc.Tab(dash2, label = 'Dash App 2', label_style = {'fontSize':'15px'}),
        dbc.Tab(dash3, label = 'Dash App 3', label_style = {'fontSize':'15px'}),
        dbc.Tab(dash4, label = 'Dash App 4', label_style = {'fontSize':'15px'}),
    ])
])

## Assemble pages

### Layout of the dashboards

In [55]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css', dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__, external_stylesheets = external_stylesheets)

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "20rem",
    "padding": "2rem 1rem",
    "background-color": "#183340"
}

# the styles for the main content position it to the right of the sidebar and add some padding.
CONTENT_STYLE = {
    "margin-left": "20rem",
    "padding": "2rem 1rem",
    "background-color":"#fcfcfc",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "right": 0,
#     "position": "fixed",
}

# the style for navigation links, to make it inline with the sidebar.
LINK_STYLE = {
    'border-radius':'5px',
    'fontSize':'15px',
    'color':'#3f5257',
    'background-color':'#fcfcfc'
}

sidebar = html.Div(
    [
        html.H2("Lebanon's Trade Performance", className="display-4", style = {'color':'#fafafa'}),
        html.Hr(style = {'border-top':'1px double #f7f7f7'}),
        dbc.Nav(
        [
            dbc.NavItem(dbc.NavLink("Home", href="/page-1", id="page-1-link",
                                    active = True, style = LINK_STYLE)),
            dbc.NavItem(dbc.NavLink("Harmonized System", href="/page-2", id="page-2-link", 
                                    active = True, style = LINK_STYLE)),
            dbc.NavItem(dbc.NavLink("User Guide", href="/page-3", id="page-3-link",
                                    active = True, style = LINK_STYLE)),
            dbc.NavItem(dbc.NavLink("Dash App", href="/page-4", id="page-4-link",
                                    active = True, style = LINK_STYLE)),            
        ],
            vertical=True, pills = True, fill = True, style = {'padding-top':'2em'}
                ),
        
        dbc.Card(dbc.CardImg(src = '/assets/aub-osb.png'),
                 body = True, style = {'height':'95px', 'position':'absolute', 'bottom':15, 'right':5, 'left':5})
    
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", style=CONTENT_STYLE)

app.layout = html.Div([dcc.Location(id="url"), sidebar, content])

#Callbacks to render content

@app.callback(
    [Output(f"page-{i}-link", "active") for i in range(1, 5)],
    [Input("url", "pathname")],
)
def toggle_active_links(pathname):
    if pathname == "/":
        # Treat page 1 as the homepage / index
        return True, False, False, False
    return [pathname == f"/page-{i}" for i in range(1, 5)]


@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname in ["/", "/page-1"]:
        return html.Div([home])
    elif pathname == "/page-2":
        return html.Div([harmonized_system])
    elif pathname == "/page-3":
        return html.Div([userguide])
    elif pathname == "/page-4":
        return html.Div([dash])

### Dash Apps Callbacks

#### Callbacks for Dash App 1

In [56]:
#Callbacks to update figures
@app.callback(Output('treemap 1_1','figure'),
             [Input('submit 1','n_clicks')],
             [State('flow 1','value'),
              State('year 1','value'),
              State('hs 1','value')])
def update_countries_treemap(n_clicks, flow, year, depth):
    fig1 = create_products_treemap(flow, year, depth)
    return fig1

@app.callback(Output('treemap 1_2','figure'),
             [Input('treemap 1_1','clickData'),
              Input('submit 1','n_clicks')],
             [State('flow 1','value'),
              State('year 1','value')])
def update_countries_treemap(clickData, n_clicks, flow, year):
    if clickData == None or clickData['points'][0]['percentRoot'] == 1:
        fig2 = create_countries_treemap(flow, year)
        return fig2
    
    product = clickData['points'][0]['label']
    dfff = df[(df['hs1_desc'] == product) | (df['hs2_desc'] == product) | 
              (df['hs4_desc'] == product) | (df['hs6_desc'] == product)]
    fig2 = create_countries_treemap(flow, year, dfff)
    return fig2

@app.callback(Output('time series 1','figure'),
             [Input('treemap 1_1','clickData'),
              Input('treemap 1_2','hoverData'),
              Input('submit 1','n_clicks')],
             [State('flow 1','value')])
def update_time_series(clickData, hoverData, n_clicks, flow):
    if (clickData == None or clickData['points'][0]['percentRoot'] == 1) and (hoverData == None or hoverData['points'][0]['percentRoot'] == 1):
        fig3 = create_time_series(flow)
        return fig3
    elif clickData != None and (hoverData == None or hoverData['points'][0]['percentRoot'] == 1):
        product = clickData['points'][0]['label']
        dfff = df[(df['hs1_desc'] == product) | 
                  (df['hs2_desc'] == product) | 
                  (df['hs4_desc'] == product) | 
                  (df['hs6_desc'] == product)]
        fig3 = create_time_series(flow, dfff)
        return fig3
    elif (clickData == None or clickData['points'][0]['percentRoot'] == 1) and hoverData != None:
        place = hoverData['points'][0]['label']
        dfff = df[(df['region'] == place) | 
                  (df['country'] == place)]
        fig3 = create_time_series(flow, dfff)
        return fig3
    else:
        product = clickData['points'][0]['label']
        dfff = df[(df['hs1_desc'] == product) | 
                  (df['hs2_desc'] == product) | 
                  (df['hs4_desc'] == product) | 
                  (df['hs6_desc'] == product)]
        
        place = hoverData['points'][0]['label']
        dfff = dfff[(dfff['region'] == place) | 
                    (dfff['country'] == place)]
        fig3 = create_time_series(flow, dfff)
        return fig3

#### Callbacks for Dash App 2

In [57]:
#Callbacks to update figures
@app.callback(Output('treemap 2_1','figure'),
             [Input('submit 2','n_clicks')],
             [State('flow 2','value'),
              State('year 2','value')])
def update_countries_treemap_2(n_clicks, flow, year):
    fig1 = create_countries_treemap(flow, year)
    return fig1

@app.callback(Output('treemap 2_2','figure'),
             [Input('treemap 2_1','clickData'),
              Input('submit 2','n_clicks')],
             [State('flow 2','value'),
              State('year 2','value'),
              State('hs 2','value')])
def update_countries_treemap(clickData, n_clicks, flow, year, depth):
    if clickData == None or clickData['points'][0]['percentRoot'] == 1:
        fig2 = create_products_treemap(flow, year, depth)
        return fig2
    place = clickData['points'][0]['label']
    dfff = df[(df['region'] == place) | 
              (df['country'] == place)]
    fig2 = create_products_treemap(flow, year, depth, dfff)
    return fig2

@app.callback(Output('time series 2','figure'),
             [Input('treemap 2_1','clickData'),
              Input('treemap 2_2','hoverData'),
              Input('submit 2','n_clicks')],
             [State('flow 2','value')])
def update_time_series(clickData, hoverData, n_clicks, flow):
    if (clickData == None or clickData['points'][0]['percentRoot'] == 1) and (hoverData == None or hoverData['points'][0]['percentRoot'] == 1):
        fig3 = create_time_series(flow)
        return fig3
    elif clickData != None and (hoverData == None or hoverData['points'][0]['percentRoot'] == 1):
        place = clickData['points'][0]['label']
        dfff = df[(df['region'] == place) | 
                  (df['country'] == place)]
        fig3 = create_time_series(flow, dfff)
        return fig3
    elif (clickData == None or clickData['points'][0]['percentRoot'] == 1) and hoverData != None:
        product = hoverData['points'][0]['label']
        dfff = df[(df['hs1_desc'] == product) | 
                  (df['hs2_desc'] == product) | 
                  (df['hs4_desc'] == product) | 
                  (df['hs6_desc'] == product)]
        fig3 = create_time_series(flow, dfff)
        return fig3
    else:
        place = clickData['points'][0]['label']
        dfff = df[(df['region'] == place) | 
                  (df['country'] == place)]
        
        product = hoverData['points'][0]['label']
        dfff = dfff[(dfff['hs1_desc'] == product) | 
                    (dfff['hs2_desc'] == product) | 
                    (dfff['hs4_desc'] == product) | 
                    (dfff['hs6_desc'] == product)]
        fig3 = create_time_series(flow, dfff)
        return fig3

#### Callbacks for Dash App 3

In [58]:
@app.callback(Output('treemap 3_1','figure'),
             [Input('submit 3','n_clicks')],
             [State('slider 3','value'),
              State('flow 3','value')])
def update_treemap_diff_products(n_clicks, value, flow):
    return build_diff_treemap_products(value, flow)

@app.callback(Output('treemap 3_2','figure'),
             [Input('treemap 3_1','clickData'),
              Input('submit 3','n_clicks')],
             [State('slider 3','value'),
              State('flow 3','value')])
def update_treemap_diff_countries(clickData, n_clicks, value, flow):
    while clickData == None or clickData['points'][0]['percentRoot'] == 1:
        return build_diff_treemap_countries(value, flow)
    product = clickData['points'][0]['label']
    
    dff = df[(df['hs1_desc']==product) |
             (df['hs2_desc']==product) |
             (df['hs4_desc']==product) |
             (df['hs6_desc']==product)]
    
    return build_diff_treemap_countries(value, flow, dff)

@app.callback(Output('time series 3','figure'),
             [Input('treemap 3_1','clickData'),
              Input('treemap 3_2','hoverData'),
              Input('submit 3','n_clicks')],
             [State('flow 3','value')])
def update_time_series(clickData, hoverData, n_clicks, flow):
    if (clickData == None or clickData['points'][0]['percentRoot'] == 1) and (hoverData == None or hoverData['points'][0]['percentRoot'] == 1):
        fig3 = create_time_series(flow)
        return fig3
    elif clickData != None and (hoverData == None or hoverData['points'][0]['percentRoot'] == 1):
        place = clickData['points'][0]['label']
        dfff = df[(df['region'] == place) | 
                  (df['country'] == place)]
        fig3 = create_time_series(flow, dfff)
        return fig3
    elif (clickData == None or clickData['points'][0]['percentRoot'] == 1) and hoverData != None:
        product = hoverData['points'][0]['label']
        dfff = df[(df['hs1_desc'] == product) | 
                  (df['hs2_desc'] == product) | 
                  (df['hs4_desc'] == product) | 
                  (df['hs1_desc'] == product)]
        fig3 = create_time_series(flow, dfff)
        return fig3
    else:
        place = clickData['points'][0]['label']
        dfff = df[(df['Region'] == place) | 
                  (df['Country'] == place)]
        
        product = hoverData['points'][0]['label']
        dfff = dfff[(dfff['hs1_desc'] == product) | 
                    (dfff['hs2_desc'] == product) | 
                    (dfff['hs4_desc'] == product) | 
                    (dfff['hs6_desc'] == product)]
        fig3 = create_time_series(flow, dfff)
        return fig3

#### Callbacks for Dash App 4

In [59]:
@app.callback(Output('treemap 4_1','figure'),
             [Input('submit 4','n_clicks')],
             [State('slider 4','value'),
              State('flow 4','value')])
def update_treemap_diff_countries_(n_clicks, value, flow):
    return build_diff_treemap_countries(value, flow)

@app.callback(Output('treemap 4_2','figure'),
             [Input('treemap 4_1','clickData'),
              Input('submit 4','n_clicks')],
             [State('slider 4','value'),
              State('flow 4','value')])
def update_treemap_diff_products_(clickData, n_clicks, value, flow):
    while clickData == None or clickData['points'][0]['percentRoot'] == 1:
        return build_diff_treemap_products(value, flow)
    place = clickData['points'][0]['label']
    
    dff = df[(df['region']==place) | 
             (df['country']==place)]
    
    return build_diff_treemap_products(value, flow, dff)
@app.callback(Output('time series 4','figure'),
             [Input('treemap 4_1','clickData'),
              Input('treemap 4_2','hoverData'),
              Input('submit 4','n_clicks')],
             [State('flow 4','value')])
def update_time_series(clickData, hoverData, n_clicks, flow):
    if (clickData == None or clickData['points'][0]['percentRoot'] == 1) and (hoverData == None or hoverData['points'][0]['percentRoot'] == 1):
        fig3 = create_time_series(flow)
        return fig3
    elif clickData != None and (hoverData == None or hoverData['points'][0]['percentRoot'] == 1):
        place = clickData['points'][0]['label']
        dfff = df[(df['region'] == place) | 
                  (df['country'] == place)]
        fig3 = create_time_series(flow, dfff)
        return fig3
    elif (clickData == None or clickData['points'][0]['percentRoot'] == 1) and hoverData != None:
        product = hoverData['points'][0]['label']
        dfff = df[(df['hs1_desc'] == product) | 
                  (df['hs2_desc'] == product) | 
                  (df['hs4_desc'] == product) | 
                  (df['hs6_desc'] == product)]
        fig3 = create_time_series(flow, dfff)
        return fig3
    else:
        place = clickData['points'][0]['label']
        dfff = df[(df['region'] == place) | 
                  (df['country'] == place)]
        
        product = hoverData['points'][0]['label']
        dfff = dfff[(dfff['hs1_desc'] == product) | 
                    (dfff['hs2_desc'] == product) | 
                    (dfff['hs4_desc'] == product) | 
                    (dfff['hs6_desc'] == product)]
        fig3 = create_time_series(flow, dfff)
        return fig3

## Run App

In [60]:
app.run_server(port = 4400)

Dash app running on http://127.0.0.1:4400/
